In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
(train_X,train_y),(test_X,test_y)=keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_X=train_X/255.0
test_X=test_X/255.0

In [ ]:
train_X[0].shape

(28, 28)

In [ ]:
train_X=train_X.reshape(len(train_X),28,28,1)
test_X=test_X.reshape(len(test_X),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_X,train_y,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.9035000205039978

Best val_accuracy So Far: 0.9129999876022339
Total elapsed time: 00h 02m 53s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        41520     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 32)                884768    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 927,578
Trainable params: 927,578
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_X, train_y, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1711 - accuracy: 0.9378 - val_loss: 0.2647 - val_accuracy: 0.9088
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1329 - accuracy: 0.9513 - val_loss: 0.2575 - val_accuracy: 0.9203
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1026 - accuracy: 0.9626 - val_loss: 0.3030 - val_accuracy: 0.9095
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0805 - accuracy: 0.9699 - val_loss: 0.3207 - val_accuracy: 0.9118
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0618 - accuracy: 0.9775 - val_loss: 0.3673 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0481 - accuracy: 0.9821 - val_loss: 0.4237 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0405 - accuracy: 0.9851 - val_loss: 0.4418 - val_accuracy

In [ ]:
model.evaluate(test_X,test_y)

313/313 [==============================] - 1s 3ms/step - loss: 0.4351 - accuracy: 0.9076


[0.43514135479927063, 0.9075999855995178]